# Megatron XinLi QA (婆媳) 预测 (BPE v2)

## CD

定位到工作目录，根据具体情况决定哦，不一定是下面的命令

In [1]:
%cd ..

/home/Public/Megatron-LM


## 环境准备

准备运行这个笔记本的 Jupyter kernel(**如果已经准备就绪，不要重复执行！**)：


1. 配置一个 Conda 环境作为 Jupyter Kernel

In [ ]:
%conda env update -f environments/environment-ipy.yml

安装完毕后，为该 Notebook 选择这个 Kernel (名为`Megatron_LM-ipy`)

2. 在Kernel所在 Conda 环境中安装 Apex

需要通过 pip 从 github 下载源代码安装：

In [ ]:
%pip install -v -r requirements/apex.txt

## 下载 Checkpoints

文件比较大，根据实际情况选择下载，**不要重复下载**

In [2]:
import os

S3_BUCKET = 'huamei'
CKPTS_DIR = 'checkpoints/345m-mildil'

S3_CKPTS_DIR = 's3://' + os.path.join(S3_BUCKET, CKPTS_DIR)

In [3]:
%%time

# 复制 latest_checkpointed_iteration.txt
!aws s3 cp \
    {S3_CKPTS_DIR} \
    {CKPTS_DIR} \
    --recursive \
    --exclude "*" \
    --include "latest_checkpointed_iteration.txt"

# 下载后读取最新的 checkpoint iter 名称
iter_step = open(f'{CKPTS_DIR}/latest_checkpointed_iteration.txt').read().strip()
ckpt_dir = f'iter_{iter_step}'

print(f'checkpoint {ckpt_dir}')

s3_ckpt_dir = os.path.join(S3_CKPTS_DIR, ckpt_dir)
local_ckpt_dir = os.path.join(CKPTS_DIR, ckpt_dir)

print(f'sync: {s3_ckpt_dir} -> {local_ckpt_dir}')
    
# 同步最新的 Checkpiont
!aws s3 sync \
    s3://huamei/hmgpt2-checkpoints/345m-hmwebmix-bpe-v2/iter_0230000 \
    ./checkpoints/345m-hmwebmix-bpe-v2/iter_0230000


checkpoint iter_230000
sync: s3://huamei/checkpoints/345m-hmwebmix-bpe-v2/iter_230000 -> checkpoints/345m-hmwebmix-bpe-v2/iter_230000
CPU times: user 27.4 ms, sys: 21.1 ms, total: 48.5 ms
Wall time: 2.26 s


## Environment Variables

- 用哪个/些 GPU?

In [3]:
%env CUDA_VISIBLE_DEVICES 0

env: CUDA_VISIBLE_DEVICES=0


## Importings

In [2]:
import copy
import csv
import json
import math
import os
import random
import sys
import time
from contextlib import closing
from itertools import chain, compress
from functools import partial
from multiprocessing import Pool
from types import SimpleNamespace

import numpy as np
import torch
import torch.nn.functional as F
from tqdm.auto import tqdm, trange

import mpu
from data_utils.tokenization import SentencePieceTokenizer, make_tokenizer
from pretrain_gpt2 import get_masks_and_position_ids
from predict_gpt2 import initialize_distributed, prepare_tokenizer, set_random_seed, setup_model, get_token_stream

## Args

In [3]:
args = SimpleNamespace(
    # Model arguments
    num_layers=24,
    hidden_size=1024,
    num_attention_heads=16,
    max_position_embeddings=1024,
    vocab_size=None,
    make_vocab_size_divisible_by=128,
    attention_dropout=0.1,
    hidden_dropout=0.1,
    # Train/valid/test data arguments.
    seq_length=1024,
    model_parallel_size=1,
    tokenizer_model_type='bert-large-uncased',
    tokenizer_type='GPT2BPETokenizer_CN',
    tokenizer_path="./data/spm/gpt2_huamei_corpus_bpe_32k_v2.model",
    cache_dir=None,
    # Training arguments.
    load='./checkpoints/345m-mildil/',
    seed=1234,
    checkpoint_activations=None,
    checkpoint_num_layers=1,
    finetune=None,
    no_load_optim=None,
    no_load_rng=None,
    resume_dataloader=None,
    fp16=True,
    hysteresis=2,
    loss_scale=None,
    loss_scale_window=1000,
    min_scale=1,
    distributed_backend='nccl',
    DDP_impl='local',
    local_rank=None,
    reset_position_ids=None,
    reset_attention_mask=None,
    eod_mask_loss=None, 
    # Text generate arguments.
    recompute=None,
    greedy=False,
    top_p=0.0,
    top_k=0,
    temperature=1.0,
#     out_seq_length=1024,
)

In [4]:
args.cuda = torch.cuda.is_available()
args.rank = int(os.getenv('RANK', '0'))
args.world_size = int(os.getenv("WORLD_SIZE", '1'))

if os.getenv('OMPI_COMM_WORLD_LOCAL_RANK'):
    # We are using (OpenMPI) mpirun for launching distributed data parallel processes
    local_rank = int(os.getenv('OMPI_COMM_WORLD_LOCAL_RANK'))
    local_size = int(os.getenv('OMPI_COMM_WORLD_LOCAL_SIZE'))

    # Possibly running with Slurm
    num_nodes = int(os.getenv('SLURM_JOB_NUM_NODES', '1'))
    nodeid = int(os.getenv('SLURM_NODEID', '0'))

    args.local_rank = local_rank
    args.rank = nodeid*local_size + local_rank
    args.world_size = num_nodes*local_size

args.model_parallel_size = min(args.model_parallel_size, args.world_size)
if args.rank == 0:
    print('using world size: {} and model-parallel size: {} '.format(
        args.world_size, args.model_parallel_size))

args.dynamic_loss_scale = False
if args.loss_scale is None:
    args.dynamic_loss_scale = True
    if args.rank == 0:
        print(' > using dynamic loss scaling')

# The args fp32_* or fp16_* meant to be active when the
# args fp16 is set. So the default behavior should all
# be false.
if not args.fp16:
    args.fp32_embedding = False
    args.fp32_tokentypes = False
    args.fp32_layernorm = False


using world size: 1 and model-parallel size: 1 
 > using dynamic loss scaling


## Init

### 初始化函数/全局变量

In [5]:
tokenizer = None
model = None

def initialize():
    global model, tokenizer

    # Disable CuDNN.
    torch.backends.cudnn.enabled = False

    # Pytorch distributed.
    initialize_distributed(args)

    # Random seeds for reproducability.
    set_random_seed(args.seed)

    # get the tokenizer
    tokenizer = prepare_tokenizer(args)

    # Model, optimizer, and learning rate.
    model = setup_model(args)

    args.device = torch.cuda.current_device()

    # setting default batch size to 1
    args.batch_size = 1

    assert mpu.get_model_parallel_rank() == 0

### 主进程初始化

In [6]:
%%time

initialize()

> initializing model parallel with size 1
> initializing model parallel cuda seeds on global rank 0, model parallel rank 0, and data parallel rank 0 with model parallel seed: 3952 and data parallel seed: 1234
prepare tokenizer done
building GPT2 model ...
 > number of parameters on model parallel rank 0: 336128000
global rank 0 is loading checkpoint ./checkpoints/345m-mildil/iter_0100000/mp_rank_00/model_optim_rng.pt
  successfully loaded ./checkpoints/345m-mildil/iter_0100000/mp_rank_00/model_optim_rng.pt
CPU times: user 11.7 s, sys: 3.65 s, total: 15.3 s
Wall time: 15.4 s


## Inference functions

In [7]:
def infer_tokens_generative(context_tokens, model, tokenizer):
    context_length = len(context_tokens)
    token_stream = get_token_stream(model, [context_tokens], tokenizer, args)   
    for i, (output_tokens, _) in enumerate(token_stream):
        if context_length + i >= args.seq_length:
            break
        ids = output_tokens.cpu().numpy().tolist()[0]
        yield ids[-1]


def infer_text_generative(contex_text, model, tokenizer):
    contex_text = contex_text.strip()
    context_tokens = tokenizer.EncodeAsIds(contex_text).tokenization
    context_length = len(context_tokens)

    token_stream = get_token_stream(model, [context_tokens], tokenizer, args)
    
    for i, (output_tokens, _) in enumerate(token_stream):
        if context_length + i >= args.seq_length:
            break
        ids = output_tokens.cpu().numpy().tolist()[0]
        s = tokenizer.DecodeIds([ids[-1]])
        yield s


## 验证是否可运行

In [8]:
input_texts = [
    (
        '刚看了一些帖子，说在怀孕的时候婆婆做的饭不好吃，或者照顾的不到位，然后媳妇都很委屈，我真的想说，你们这些婆婆在我看来都是天使，什么叫JP的婆家（不是某一个人，而是一家）看看我的遭遇就知道了。我婆婆在我怀孕初期反应剧烈的时候说她没空（其实除了伺候她女儿一家，就是在家看肥皂剧、明星绯闻什么的，这些她样样精通）不能过来照顾我，因为我当时非常严重一个月瘦了十几斤，贫血贫的连床都下不了，在我老公的登门请求下，她三个月内总共给我们弄过2次菜，一次是毒蘑菇，吃的我夜里一点吐血进医院，另一次是死鱼，吃完以后我和老公都拉肚子一个星期，这可都是我怀孕期间她干的事啊，医生严重警告了孕妇前三个月拉肚子很可能导致流产。我老公找她问她为什么要买这些坏掉的菜给孕妇吃，为什么连孙子都不管不顾，她还气的要命呢，说，早说了我没空，我哪有功夫管你们，你们想吃好的自己买去吧。对了，这两样菜是我老公给了1000块钱给他妈才买来的呢。本来是觉得不放心保姆一个人在家照顾我才百般央求婆婆能念在亲情的份上稍微照顾我一点，可惜发现她真的比一个见死不救的路人还要冷血。',
        '介绍一下我的婆家，公公婆婆都退休在家，收入可靠，颇有积蓄，但是从我们买房、装修到结婚前后的诸多事件中，他们从来不花一分钱，没出过一点力，任何礼节、规矩，他们全部都假装不知道，从定亲到结婚，所有要用到的大小花销，大到订婚的礼钱，小到连结婚用到的一个囍字、一颗糖，全部都是我老公拿钱出来，并且自己亲自买好的。为了忙结婚我老公瘦了十几斤，但他爸妈从来不闻不问，那他们每天在家干什么呢？我的公公每天一大早就出去打牌，一直打到晚上吃完宵夜才回家。婆婆忙着伺候她年近40身体健康的女儿全家，还要忙着追电视剧，研究明星绯闻。我们装修时进材料请求公公来帮忙看一天，他11点终于到了，11点半就打电话给我老公让他快过来，我老公不知道发生了什么事赶快回家，结果是问我们中午安排他在哪家饭店吃饭，结果那天中午他一个人喝掉一斤白酒，然后在饭店就说：“累了，我回家睡觉了”然后不管不顾走了，感情我们花了300多块钱请他来专门吃饭的，后来没办法，我只好请了半天假在家看着。最让人心寒的是结婚前特别忙的一段时间，因为我老公每天回家都很晚，公婆居然让我老公搬出去住，说是影响他们休息了。蘑菇整个伞面里面全部都长满了一厘米左右长的黑霉，够可怕的！',
    ),
    
]

In [9]:
# args.recompute=True
# args.top_p=0.0
# args.top_k=0
# args.temperature=1.0

n_gen = 2

for question_title, question_text in input_texts:
    text = question_title.strip() + '<sep>' \
        + question_text.strip()  + '<sep>' \
        + '<sep><sep><|endoftext|>'
    print(text)
    print()
    for i in range(n_gen):
#         args.temperature=random.gauss(0.95, 0.05)
        print(f'{i}: ', end='')
        s_pred = ''
        for s in infer_text_generative(text, model, tokenizer):
            print(s, end='')
        print(os.linesep)
    print()
    print('=' * 100)
    print()
    

刚看了一些帖子，说在怀孕的时候婆婆做的饭不好吃，或者照顾的不到位，然后媳妇都很委屈，我真的想说，你们这些婆婆在我看来都是天使，什么叫JP的婆家（不是某一个人，而是一家）看看我的遭遇就知道了。我婆婆在我怀孕初期反应剧烈的时候说她没空（其实除了伺候她女儿一家，就是在家看肥皂剧、明星绯闻什么的，这些她样样精通）不能过来照顾我，因为我当时非常严重一个月瘦了十几斤，贫血贫的连床都下不了，在我老公的登门请求下，她三个月内总共给我们弄过2次菜，一次是毒蘑菇，吃的我夜里一点吐血进医院，另一次是死鱼，吃完以后我和老公都拉肚子一个星期，这可都是我怀孕期间她干的事啊，医生严重警告了孕妇前三个月拉肚子很可能导致流产。我老公找她问她为什么要买这些坏掉的菜给孕妇吃，为什么连孙子都不管不顾，她还气的要命呢，说，早说了我没空，我哪有功夫管你们，你们想吃好的自己买去吧。对了，这两样菜是我老公给了1000块钱给他妈才买来的呢。本来是觉得不放心保姆一个人在家照顾我才百般央求婆婆能念在亲情的份上稍微照顾我一点，可惜发现她真的比一个见死不救的路人还要冷血。<sep>介绍一下我的婆家，公公婆婆都退休在家，收入可靠，颇有积蓄，但是从我们买房、装修到结婚前后的诸多事件中，他们从来不花一分钱，没出过一点力，任何礼节、规矩，他们全部都假装不知道，从定亲到结婚，所有要用到的大小花销，大到订婚的礼钱，小到连结婚用到的一个囍字、一颗糖，全部都是我老公拿钱出来，并且自己亲自买好的。为了忙结婚我老公瘦了十几斤，但他爸妈从来不闻不问，那他们每天在家干什么呢？我的公公每天一大早就出去打牌，一直打到晚上吃完宵夜才回家。婆婆忙着伺候她年近40身体健康的女儿全家，还要忙着追电视剧，研究明星绯闻。我们装修时进材料请求公公来帮忙看一天，他11点终于到了，11点半就打电话给我老公让他快过来，我老公不知道发生了什么事赶快回家，结果是问我们中午安排他在哪家饭店吃饭，结果那天中午他一个人喝掉一斤白酒，然后在饭店就说：“累了，我回家睡觉了”然后不管不顾走了，感情我们花了300多块钱请他来专门吃饭的，后来没办法，我只好请了半天假在家看着。最让人心寒的是结婚前特别忙的一段时间，因为我老公每天回家都很晚，公婆居然让我老公搬出去住，说是影响他们休息了。蘑菇整个伞面里面全部都长满了一厘米左右长的黑霉，够可怕的！<sep><sep><sep><|endoftex

## Test

使用 test 语料，从中随机打断，并预测下文，比较原文与预测结果！

随机选 N 个

In [14]:
input_file = './data/mildil/mildil.test.json'

with open(input_file) as fp:
    for line in fp:
        data = json.loads(line)
        text = data['text']
for x in text.split('<sep><sep><|endoftext|>'):
    print(x)
    print()

之所以发这个帖子是因为之前婆婆一家有太多奇葩事迹，比电视剧还狗血，我这个记性又不好，也不是记性不好，就是心大，不太容易把苦啊那些记在心里，而且很容易别人对我一点好我就把之前的不好都忘了，所以这次婆婆来我家住一个月我准备以血日记的形式记录一下。真的，编剧们可以看看，说不定有启发呢。背景：我和老公结婚3年，备孕1年，居住海外，我全职在家，婆婆此次因为自己的私事来呆一个月。现在已经第6天了，所以前6天简单回溯一下第1天：抵达，老公接机，凌晨2点，我做了粥，炒了两个菜，等着婆婆回来吃。第2天：收拾。第3天：带来了很多瓶瓶罐罐，把调味盒子全部更换了，我才是下厨的人，有些盒子好看是好看，可是调味料根本弄不起来啊。这事我一句没有多提，我想她大老远背过来也不容易。第4天：这天是每周我们洗衣服的日子，我和朋友有约，早早出门了，但鉴于太爱“帮忙”的性格，我出门告诉她千万别帮我们洗衣服，她膝盖不好，洗衣筐要扛到洗衣间确实比较重。（这里介绍下我婆婆是属于特别喜欢沉醉于牺牲奉献的一个人，特别喜欢苦情的戏码。你越是让她不做的事她偏偏要做，让你觉得欠她很多，但这些付出都不是必要的。比如她对紫外线过敏，不能晒太阳，我们去逛街，我在国外不好意思撑伞遮阳就随手拿着那些广告纸宣传纸什么的遮一下，我公公以前有腰肌劳损，我从来没见者犯病过，我就说公公帮妈拿一点购物袋吧，我手上已经很多了，她却说“你爸腰不好”然后一个人大包小包穿梭在阳光里。就是有阳光的地方她就举起挂着各种购物袋有宣传纸打的手挡住阳光，然后跑几步到阴凉的地方，而公公在那儿闲庭信步。所以每次我们从超市采购回家，公公都是一个拎着自己的那包东西，一般是零食种子什么的，而我们其他人大包小包，害的跑几趟）我回家，一进车库，车库里面晾着全是内裤。回到家，家里的摆设也随她的要求变换了位置。我有点生气，觉得她对我们的关爱有点太没有界限了，但也觉得她也是好心就没说什么。回家之后她反而提起今天做的事，我说“我回来之后没说话就是因为不想鼓励你太为我们受累了，我们是想让你多休息”她说“我在家我也不可能不动吧，而且有时差，做事反而分散注意力，不那么想睡觉。我也没说什么了。晚上，老公回家，她进房间睡觉之前和我老公说“好无聊啊，赶快把事情办完我回国了”这只是过了一天我平时天天过的日子啊。我呢，很天真的跟老公说了些洗内裤的事，他没什么回应，然后他心理念着他妈刚刚说要想赶快回国

In [20]:
n_samples = 1000
b_random = False
b_shuffle = False

input_file = './data/mildil/mildil.test.json'
output_file = f'./data/mildil/mildil.test_{n_samples}-{iter_step}-{args.seq_length}.tsv'

print(f'output_file={output_file}')

total = sum(1 for _ in tqdm(open(input_file)))
print(f'Test 数据总数: {total:,d}')

if n_samples > 0:
    assert total >= n_samples
else:
    n_samples = total

print(f'Test 采样数: {n_samples:,d}')

mask = np.zeros(total, dtype=int)
mask[:n_samples] = 1
if b_random:
    np.random.shuffle(mask)

samples = []
with open(input_file) as fp:
    reader = compress(fp, mask)
    for line in tqdm(reader, 'sample', total=N):
        line = line.strip()
        if not line:
            continue
        text = json.loads(line)['text']
        samples.append(text)

if b_shuffle:
    random.shuffle(samples)

delimiter = '<sep><sep><|endoftext|>'

with open(output_file, 'w') as fp:
    writer = csv.writer(fp, delimiter='\t')
    writer.writerow(['标题', '内容', '回答', '生成文本'])
    for text in tqdm(samples, 'infer'):
        question, answer = text.split(delimiter)
        question_title, question_text = question.split('<sep>')
        input_txt = question + delimiter
        infer_txt = ''.join(id_ for id_ in infer_text_generative(input_txt, model, tokenizer))
        row = [question_title, question_text, answer, infer_txt]
        writer.writerow(row)


output_file=./data/mildil/mildil.test_10-100000-1024.tsv



Test 数据总数: 1,000
Test 采样数: 1,000
